In [1]:
import math
import numpy as np

In [2]:
class Artillery:
    def __init__(self, mass, air_drag, initial_velocity):
        self.mass = mass
        self.air_drag = air_drag
        self.initial_velocity = initial_velocity


m252 = Artillery(0.25, 0.00031, 375)
m119 = Artillery(23, 0.0043, 212.5)

In [3]:
class GridCoord:
    def __init__(self, easting: float, northing: float):
        self.easting = easting
        self.northing = northing

In [4]:
def deg2EL(degrees):
    return math.radians(degrees) * 1000 - 960


def distancefunc(x1, y1, x2, y2):
    return math.sqrt(math.pow((x2 - x1), 2) + math.pow((y2 - y1), 2))

In [5]:
def rotate_point_clockwise(point, deg_rotation):
    rad_rotation = math.radians(deg_rotation)
    rotation_matrix = np.array(
        [
            [math.cos(rad_rotation), math.sin(rad_rotation)],
            [-math.sin(rad_rotation), math.cos(rad_rotation)],
        ]
    )
    return np.dot(rotation_matrix, point)


# Rotate around the centroid of the polygon (average of all points)
def rotate_polygon_clockwise(points, deg_rotation):
    total_x = 0
    total_y = 0
    for point in points:
        total_x = total_x + point[0]
        total_y = total_y + point[0]
    center = vec([total_x, total_y]) / len(points)
    # Now subtract the center point to get the shape centered about origin
    centered_points = [point - center for point in points]
    centered_points = [
        rotate_point_clockwise(point, deg_rotation) for point in centered_points
    ]
    return [point + center for point in centered_points]

In [6]:
def ballistic_sim(artillery, elevation, target_height):
    """Calculates the distance and time of flight for a projectile launched
    with `initial_velocity` m/s, `elevation` degrees, landing at `target_height`
    relative height from launch height, with simulation timestep of `delta_t`
    """
    delta_t = 0.001
    include_drag = True  # False to just use gravity
    f = artillery.air_drag
    m = artillery.mass
    initial_velocity = artillery.initial_velocity
    t = 0
    position = np.array([0, 0])
    gravity = np.array([0, -9.81])
    velocity = initial_velocity * np.array(
        [math.cos(math.radians(elevation)), math.sin(math.radians(elevation))]
    )
    while velocity[1] > 0 or position[1] > target_height:
        speed = np.linalg.norm(velocity)
        drag_speed = f * speed * speed / m
        drag_direction = -1 * (velocity / speed)
        drag_vector = drag_speed * drag_direction
        deceleration = drag_vector + gravity if include_drag else gravity

        velocity = velocity + deceleration * delta_t
        position = position + velocity * delta_t
        t = t + delta_t
    return position[0], t

In [7]:
# FIXME: Not taking into account relative height from observer, which means I'm
# calculating eastings/northings based on hypotenuse of the triangle rather
# than the actual x/y distance
def azimuth_from_observer_relative(
    mortar_pos: GridCoord,
    observer_pos: GridCoord,
    obs_to_enemy_azimuth: float,
    obs_to_enemy_distance: float,
):
    # First need to calculate easting and northing of enemy
    # relative to observer
    obs_to_enemy_pos = GridCoord(0, 0)

    # Due North
    if obs_to_enemy_azimuth == 0:
        obs_to_enemy_pos.easting = 0
        obs_to_enemy_pos.northing = obs_to_enemy_distance
    # First Quadrant
    elif obs_to_enemy_azimuth < 90:
        angle = 90 - obs_to_enemy_azimuth
        obs_to_enemy_pos.easting = math.cos(math.radians(angle)) * obs_to_enemy_distance
        obs_to_enemy_pos.northing = (
            math.sin(math.radians(angle)) * obs_to_enemy_distance
        )
    # Due East
    elif obs_to_enemy_azimuth == 90:
        obs_to_enemy_pos.easting = obs_to_enemy_distance
        obs_to_enemy_pos.northing = 0
    # Second Quadrant
    elif obs_to_enemy_azimuth < 180:
        angle = obs_to_enemy_azimuth - 90
        obs_to_enemy_pos.easting = math.cos(math.radians(angle)) * obs_to_enemy_distance
        obs_to_enemy_pos.northing = (
            -1 * math.sin(math.radians(angle)) * obs_to_enemy_distance
        )
    # Due South
    elif obs_to_enemy_azimuth == 180:
        obs_to_enemy_pos.easting = 0
        obs_to_enemy_pos.northing = -1 * obs_to_enemy_distance

    # Third Quadrant
    elif obs_to_enemy_azimuth < 270:
        angle = 270 - obs_to_enemy_azimuth
        obs_to_enemy_pos.easting = (
            -1 * math.cos(math.radians(angle)) * obs_to_enemy_distance
        )
        obs_to_enemy_pos.northing = (
            -1 * math.sin(math.radians(angle)) * obs_to_enemy_distance
        )
    # Due West
    elif obs_to_enemy_azimuth == 270:
        obs_to_enemy_pos.easting = -1 * obs_to_enemy_distance
        obs_to_enemy_pos.northing = 0
    # Fourth Quadrant
    else:
        angle = obs_to_enemy_azimuth - 270
        obs_to_enemy_pos.easting = (
            -1 * math.cos(math.radians(angle)) * obs_to_enemy_distance
        )
        obs_to_enemy_pos.northing = (
            math.sin(math.radians(angle)) * obs_to_enemy_distance
        )

    enemy_pos = GridCoord(
        observer_pos.easting + obs_to_enemy_pos.easting,
        observer_pos.northing + obs_to_enemy_pos.northing,
    )

    distance = distancefunc(
        mortar_pos.easting, mortar_pos.northing, enemy_pos.easting, enemy_pos.northing
    )

    return (
        azimuth_from_grids(mortar_pos, enemy_pos),
        distance,
    )


def azimuth_from_grids(start_pos: GridCoord, end_pos: GridCoord):
    start_end_vector = GridCoord(
        end_pos.easting - start_pos.easting, end_pos.northing - start_pos.northing
    )

    # since we want azimuth, we need to shift the axes
    # east becomes north, and north becomes west

    temp = start_end_vector.northing
    start_end_vector.northing = start_end_vector.easting
    start_end_vector.easting = temp

    initial = math.degrees(
        math.atan2(start_end_vector.northing, start_end_vector.easting)
    )
    if initial < 0:
        initial = 360 + initial

    return initial


def calculate_elevation(
    artillery: Artillery, target_height: float, distance: float, upper_half: bool
):
    # Check if target is even reachable using optimal launch angle
    current_elevation = 45
    current_distance = ballistic_sim(artillery, current_elevation, target_height)[0]
    if current_distance < distance:
        # target unreachable
        return "Target Unreachable."
    # Good Ole' Binary Search :)
    if upper_half:
        max_elevation = 90
        min_elevation = 45
    else:
        max_elevation = 45
        min_elevation = 0
    while abs(current_distance - distance) > 1:
        current_elevation = (min_elevation + max_elevation) / 2
        current_distance = ballistic_sim(artillery, current_elevation, target_height)[0]
        if upper_half:
            if current_distance < distance:
                max_elevation = current_elevation
            else:
                min_elevation = current_elevation
    return {
        "elevation": round(current_elevation, 1),
        "time_to_impact": round(
            ballistic_sim(artillery, current_elevation, target_height)[1],
            1,
        ),
    }

In [8]:
def generate_ballistics_table(artillery: Artillery, heights=None):
    if heights == None:
        heights = [height for height in range(0, -10000, -100)]
    else:
        heights = [-1 * height for height in heights]
    for height in heights:
        result = ballistic_sim(artillery, 0, height)
        print("x: ", result[0], "y: -", height, "z: ", result[1])

In [9]:
reforger_tools_drops = [
    16.672001,
    37.368,
    65.996002,
    102.334999,
    123.328003,
    146.169006,
    170.830002,
    197.285004,
    225.509995,
    255.477997,
    287.165985,
    320.548004,
    355.600006,
    392.298004,
    430.618011,
    470.536987,
    512.031006,
    555.078979,
    599.656006,
    645.742004,
    693.315002,
    742.348999,
    792.828003,
    844.728027,
    898.026001,
    952.70697,
    1008.742981,
    1066.119019,
    1124.812988,
    1184.807983,
    1246.081055,
    1277.192993,
    1308.61499,
    1340.348022,
    1372.38501,
    1404.733032,
    1437.383057,
    1470.332031,
    1503.577026,
    1537.119995,
    1570.958008,
    1605.088013,
    1639.505981,
    1674.207031,
    1709.199951,
    1744.469971,
    1780.021973,
    1815.850952,
    1851.959961,
    1888.333984,
    1924.989014,
    1961.911011,
    1999.089966,
    2036.546021,
    2074.261963,
    2112.233887,
    2150.462891,
    2188.954102,
    2227.699951,
    2266.697021,
    2305.940918,
    2345.435059,
    2385.180908,
    2425.159912,
    2465.388916,
    2505.846924,
    2546.559082,
    2587.493896,
    2628.670898,
    2670.074951,
    2711.705078,
    2753.579102,
    2795.662109,
    2837.971924,
    2880.51001,
    2923.27002,
    2966.239014,
    3009.429932,
    3052.831055,
    3096.447021,
    3140.263916,
    3184.310059,
    3228.552979,
    3272.98999,
    3317.649902,
    3362.5,
    3407.548096,
    3452.795898,
    3498.24292,
    3543.878906,
    3589.702881,
    3635.727051,
    3681.937988,
    3728.331055,
    3774.907959,
    3821.674072,
    3868.616943,
    3915.731934,
    3963.039062,
    4010.506104,
    4058.163086,
    4105.987793,
    4153.984863,
    4202.146973,
    4250.487793,
    4298.966797,
    4347.626953,
    4396.454102,
    4445.434082,
    4494.575195,
    4543.878906,
    4593.331055,
    4642.938965,
    4692.696777,
    4742.609863,
    4792.666016,
    4842.868164,
    4893.221191,
    4943.708984,
    4994.356934,
    5045.134766,
    5096.048828,
    5147.094238,
    5198.279785,
    5223.944824,
    5249.606934,
    5301.062988,
    5352.658203,
    5404.36084,
    5456.212891,
    5508.169922,
    5560.282227,
    5612.487793,
    5664.821777,
    5717.266113,
    5769.845215,
    5822.554199,
    5875.347168,
    5928.262207,
    5981.300781,
    6034.458008,
    6087.696777,
    6141.054199,
    6194.508789,
    6248.080078,
    6274.911133,
    6301.745117,
    6355.512207,
    6409.395996,
    6463.367188,
    6517.424805,
    6571.588867,
    6598.713867,
    6625.838867,
    6680.192871,
    6734.625977,
    6789.143066,
    6816.444824,
    6871.108887,
    6925.845215,
    6953.23584,
    7008.10791,
    7063.030762,
    7090.568848,
    7118.070801,
    7173.173828,
    7200.762207,
    7256,
    7283.61377,
    7338.937988,
    7366.652832,
    7394.353027,
    7449.835938,
    7505.373047,
    7560.977051,
    7588.811035,
    7616.634766,
    7672.399902,
    7728.21582,
    7784.083008,
    7840.027832,
    7896.005859,
    7924.032227,
    7952.045898,
    8008.150879,
    8036.243164,
    8064.330078,
    8120.519043,
    8148.647949,
    8176.780762,
    8204.944336,
    8233.103516,
    8289.466797,
    8345.873047,
    8374.139648,
    8402.34082,
    8458.867188,
    8515.391602,
    8543.688477,
    8571.991211,
    8600.298828,
    8656.950195,
    8713.626953,
    8741.986328,
    8798.723633,
    8827.123047,
    8855.493164,
    8883.899414,
    8912.291992,
    8940.71875,
    8969.149414,
    9026.033203,
    9054.472656,
    9082.914062,
    9111.380859,
    9139.84668,
    9168.333984,
    9196.817383,
    9253.785156,
]

# generate_ballistics_table(375, 0, reforger_tools_drops)

In [10]:
mortar_pos = GridCoord(3273, 2495)
obs_pos = GridCoord(2487, 3019)
obs_azimuth = 322
obs_distance = 366
target_height = -32

res = azimuth_from_observer_relative(
    mortar_pos,
    obs_pos,
    obs_azimuth,
    obs_distance,
)
print(res)
print(deg2EL(calculate_elevation(m252, target_height, res[1], True)["elevation"]))

(308.7752181763813, 1297.2300373838832)
6.912405604858577


In [11]:
# import matplotlib
# import matplotlib.pyplot as plt

# for angle in range(0, 45):
#     plot_points = ballistic_sim(375, angle, 0, 0.001, True)[2]
#     x = [point[0] for point in plot_points]
#     y = [point[1] for point in plot_points]
#     x = x[::10]
#     y = y[::10]
#     myplot = plt.plot(x, y)
#     ax = plt.gca()
#     ax.set_xlim(0, 5000)
#     plt.show()